In [1]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.

每个 nn.Module 的子类都在 forward 方法里实现对输入数据的运算。



In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()#创建展平层：把每张 28×28 图像变成长度 784 的向量（保留 batch 维）
        self.linear_relu_stack = nn.Sequential(#用 Sequential 把多层按顺序串起来，形成一个“前馈 MLP”：
            nn.Linear(28*28, 512),#nn.Linear(28*28, 512)：输入 784 维 → 输出 512 维
            nn.ReLU(),#nn.ReLU()：激活函数，引入非线性
            nn.Linear(512, 512),#nn.Linear(512, 512)：512 → 512
            nn.ReLU(),#nn.ReLU()：再一次非线性
            nn.Linear(512, 10),#nn.Linear(512, 10)：512 → 10（10 类的打分）
        )

    def forward(self, x):
        x = self.flatten(x)#把 [batch, 28, 28] 变成 [batch, 784]
        logits = self.linear_relu_stack(x)#送入 MLP，输出 [batch, 10] 的“logits”（未归一化分数）
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [5]:
X = torch.rand(1, 28, 28, device=device)#造一个随机“假图片”：batch=1，大小 28×28，并放到同一个 device。
logits = model(X)#前向传播，得到形状 [1, 10] 的 logits。
#用 model(X) 才会触发 PyTorch 的钩子（hooks）、自动混合精度、train()/eval() 行为等机制；直接 model.forward(X) 可能绕开这些。
pred_probab = nn.Softmax(dim=1)(logits)
#对类别维（dim=1）做 softmax，把每行 logits 变成和为 1 的概率分布。
#注意这里写法是“先创建模块 nn.Softmax(dim=1)，再把 logits 当函数调用进去”。

y_pred = pred_probab.argmax(1)
#在 dim=1 上取最大概率的索引，得到预测类别 id（形状 [1]）

print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [6]:
input_image = torch.rand(3,28,28)#生成 batch=3 的随机图像张量。
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [7]:
flatten = nn.Flatten()#flatten = nn.Flatten()：创建展平层
flat_image = flatten(input_image)#把 [3,28,28] 变成 [3,784]。
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)#定义一层全连接，把 784 维映射到 20 维。
hidden1 = layer1(flat_image)#输入 [3,784] 输出 [3,20]。
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2276,  0.1369, -0.4970, -0.2451,  0.3679, -0.0870,  0.8411,  0.1284,
          0.0930, -0.1318,  0.5965,  0.0984, -0.3074, -0.2858,  0.2364,  0.0869,
          0.3232,  0.8629,  0.0453, -0.5829],
        [ 0.3516,  0.0743, -0.7042, -0.0764, -0.2121, -0.0791,  0.8915,  0.2704,
         -0.0100, -0.4283,  0.4936, -0.2967, -0.1486, -0.4171,  0.2897,  0.1940,
          0.1536,  0.6228, -0.0866, -0.2914],
        [ 0.1924,  0.0420, -0.5317, -0.3872,  0.6132, -0.0639,  0.5986,  0.2517,
          0.1323, -0.1308,  0.4074,  0.0629, -0.3168, -0.6686,  0.2723, -0.0076,
          0.1556,  0.7166, -0.2342, -0.1924]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2276, 0.1369, 0.0000, 0.0000, 0.3679, 0.0000, 0.8411, 0.1284, 0.0930,
         0.0000, 0.5965, 0.0984, 0.0000, 0.0000, 0.2364, 0.0869, 0.3232, 0.8629,
         0.0453, 0.0000],
        [0.3516, 0.0743, 0.0000, 0.0000, 0.0000, 0.0000, 0.8915, 0.2704, 0.0000,
         0.0000, 0.4936, 0.0000, 0.0000, 0.0000, 0.28

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [11]:
seq_modules = nn.Sequential(#把 flatten → layer1 → ReLU → Linear(20→10) 串起来。
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)#新的随机输入 batch
logits = seq_modules(input_image)#直接把输入喂给容器，内部会按顺序执行；输出形状 [3,10]。

In [12]:
print(logits)

tensor([[-0.3695,  0.4914, -0.1291, -0.3216,  0.0303,  0.1590, -0.1167, -0.1650,
         -0.1205,  0.2080],
        [-0.2133,  0.3717, -0.0070, -0.1212, -0.0369,  0.0652, -0.0388, -0.1756,
         -0.0768,  0.2160],
        [-0.4871,  0.4133, -0.1658, -0.3069,  0.0897,  0.1938, -0.0667, -0.3666,
         -0.1717,  0.4016]], grad_fn=<AddmmBackward0>)


nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [14]:
softmax = nn.Softmax(dim=1)#dim=1：对类别维做 softmax（每个样本一行概率和为 1）。
pred_probab = softmax(logits)#pred_probab：形状仍是 [batch, 10]，但值变为概率。

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
    #param[:2]：只打印前两行/前两项做预览，避免输出过大。

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0243,  0.0139, -0.0207,  ...,  0.0072, -0.0102,  0.0117],
        [-0.0262,  0.0222,  0.0099,  ..., -0.0134,  0.0111, -0.0049]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0207, -0.0256], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0293,  0.0150, -0.0065,  ...,  0.0041, -0.0105,  0.0401],
        [ 0.0033,  0.0305,  0.0249,  ...,  0.0116,  0.0006,  0.0249]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
